In [1]:
import argparse
import os
import sys
import pickle
import time
import subprocess
import shutil
sys.path.append(os.getcwd())

from khrylib.utils import *
from khrylib.rl.utils.visualizer import Visualizer
from khrylib.rl.core.policy_gaussian import PolicyGaussian
from khrylib.rl.core.critic import Value
from khrylib.models.mlp import MLP
from motion_imitation.envs.humanoid_im import HumanoidEnv, HumanoidEnvFreezeKnee
from motion_imitation.utils.config import Config
import matplotlib.pyplot as plt
import glfw




In [2]:
cfg = Config('0202', False, create_dirs=False)
cfg_f = Config('0202_freeze', False, create_dirs=False)
cfg.env_start_first = True
cfg_f.env_start_first = True
logger = create_logger(os.path.join(cfg.log_dir, 'log_eval.txt'))

"""make and seed env"""
dtype = torch.float64
torch.set_default_dtype(dtype)
torch.manual_seed(cfg.seed)
torch.set_grad_enabled(False)
env = HumanoidEnv(cfg)
env.seed(cfg.seed)


body_qposaddr = get_body_qposaddr(env.model)
body_qposaddr_list_start_index = [idxs[0] for idxs in list(body_qposaddr.values())]
knee_idx = body_qposaddr_list_start_index[2]
def plot_qpos(qpos, body_qposaddr_list_start_index):
    fig, axs = plt.subplots(nrows=qpos.shape[1]//4+1, ncols=4, figsize=(10, 12))
    for i in range(qpos.shape[1]//4+1):
        for j in range(4):
            idx = i*4 + j
            if idx >= qpos.shape[1]:
                break
            gt = qpos[:, idx]
            axs[i, j].plot(gt, 'r', label='gt')
            axs[i, j].set_ylim([-np.pi, np.pi])
            if idx in body_qposaddr_list_start_index:
                body_name = [name for name, addr in body_qposaddr.items() if addr[0] == idx][0]
                axs[i, j].set_title(f"idx = {idx}, {body_name}", fontsize=12)
            if i == 0 and j == 0:
                axs[i, j].legend()
    plt.tight_layout()
    plt.show()

# Example usage:
# plot_qpos(qpos, body_qposaddr_list_start_index )



env_f = HumanoidEnvFreezeKnee(cfg_f, cfg)
env_f.seed(cfg_f.seed)

actuators = env.model.actuator_names
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

actuators_f = env_f.model.actuator_names
state_dim_f = env_f.observation_space.shape[0]
action_dim_f = env_f.action_space.shape[0]

"""load learner policy"""
policy_net = PolicyGaussian(MLP(state_dim, cfg.policy_hsize, cfg.policy_htype), action_dim, log_std=cfg.log_std, fix_std=cfg.fix_std)
value_net = Value(MLP(state_dim, cfg.value_hsize, cfg.value_htype))
cp_path = '%s/iter_%04d.p' % (cfg.model_dir, 800)
logger.info('loading model from checkpoint: %s' % cp_path)
model_cp = pickle.load(open(cp_path, "rb"))
policy_net.load_state_dict(model_cp['policy_dict'])
value_net.load_state_dict(model_cp['value_dict'])
running_state = model_cp['running_state']


dim of qpos, qvel = ((39,), (38,))


loading model from checkpoint: results/motion_im/0202/models/iter_0800.p


In [3]:
class MyVisulizerFK(Visualizer):

    def __init__(self, vis_file):
        super().__init__(vis_file)
        ngeom = len(env.model.geom_rgba) - 1
        self.env_vis.model.geom_rgba[ngeom + 21: ngeom * 2 + 21] = np.array([0.7, 0.0, 0.0, 1])
        self.env_vis.viewer.cam.lookat[2] = 1.0
        self.env_vis.viewer.cam.azimuth = 45
        self.env_vis.viewer.cam.elevation = -8.0
        self.env_vis.viewer.cam.distance = 5.0
        self.T = 12

    def data_generator(self):
        poses = {'pred': [], 'gt': []}
        state = env_f.reset()
        # print(env_f.init_qpos[1], env.expert['qpos'][0,1])
        assert env_f.init_qpos_f[1] == env.expert['qpos'][0,1], "init_qpos[1] != expert_qpos[0,1]"
        assert np.allclose(env_f.init_qpos_f, env_f.data.qpos), "init_qpos[1] != qpos[0,1]"

        # state_f_qpos, state_f_qvel = state_f[:env_f.data.qpos.shape[0]], state_f[env_f.data.qpos.shape[0]:]
        # state_old = np.concatenate([state_f_qpos[:knee_idx ], np.zeros(4),state_f_qpos[knee_idx:], state_f_qvel[:knee_idx ], np.zeros(4), state_f_qvel[knee_idx:], 0.*np.ones(1)])
        if running_state is not None:
            state = running_state(state, update=False)
        for t in range(75): 
            
            epos = env.get_expert_attr('qpos', env.get_expert_index(t)).copy()
            # print(epos.shape)
            if env.expert['meta']['cyclic']:
                init_pos = env.expert['init_pos']
                cycle_h = env.expert['cycle_relheading'] # relative heading quaternion
                cycle_pos = env.expert['cycle_pos']
                epos[:3] = quat_mul_vec(cycle_h, epos[:3] - init_pos) + cycle_pos 
                epos[3:7] = quaternion_multiply(cycle_h, epos[3:7])
            poses['gt'].append(epos) 
            qpos = env_f.data.qpos.copy()
            qpos = np.concatenate([qpos[:knee_idx ], np.zeros(4),qpos[knee_idx:]])
            poses['pred'].append(qpos)
            
            state_var = tensor(state, dtype=dtype).unsqueeze(0)
            action = policy_net.select_action(state_var, mean_action=True)[0].cpu().numpy()
            # print(t, state.shape, action.shape)
            next_state, reward, done = env_f.step(action)
            # print(np.std(next_state_f))
            if running_state is not None:
                next_state = running_state(next_state, update=False)
            if done:
                # print(f"fail: {info['fail']}")
                break
            state = next_state
            # print("np.std(state) vs ",np.std(state), np.std(epos))

        poses['gt'] = np.vstack(poses['gt'])
        poses['pred'] = np.vstack(poses['pred'])
        plot_pose = False
        if plot_pose:
            # import matplotlib.pyplot as plt
            fig, axs = plt.subplots(nrows=poses['gt'].shape[1]//4+1, ncols=4, figsize=(10, 12))
            for i in range(poses['gt'].shape[1]//4+1):
                for j in range(4):
                    if i*4 + j < poses['gt'].shape[1]:
                        gt = poses['gt'][:, i*4 + j]
                        pred = poses['pred'][:, i*4 + j]
                        mse = np.mean((gt - pred) ** 2)
                        axs[i, j].plot(gt, 'r', label='gt')
                        axs[i, j].plot(pred, 'b', label='pred')
                        axs[i, j].set_ylim([-np.pi, np.pi])
                        axs[i, j].set_title(f'{i*4 + j} MSE: {mse:.4f}')
                    if i == 0 and j == 0:
                        axs[i, j].legend()
            plt.tight_layout()
            plt.show()
        self.num_fr = poses['pred'].shape[0]
        yield poses
            

    def update_pose(self):
        self.env_vis.data.qpos[:env.model.nq] = self.data['pred'][self.fr]
        self.env_vis.data.qpos[env.model.nq:] = self.data['gt'][self.fr] 
        self.env_vis.data.qpos[env.model.nq] += 1.0 # time update
        # if args.record_expert:
        #     self.env_vis.data.qpos[:env.model.nq] = self.data['gt'][self.fr]
        if True: #args.hide_expert:
            self.env_vis.data.qpos[env.model.nq + 2] = 100.0
        # if args.focus:
        #     self.env_vis.viewer.cam.lookat[:2] = self.env_vis.data.qpos[:2]
        self.env_vis.sim_forward()

    def record_video(self):
        frame_dir = f'out/videos/freeze_knee/frames'
        if os.path.exists(frame_dir):
            shutil.rmtree(frame_dir)
        os.makedirs(frame_dir, exist_ok=True)
        for fr in range(self.num_fr):
            # print("hi")
            self.fr = fr
            self.update_pose()
            for _ in range(200): 
                self.render()
            if True: # not args.preview:
                t0 = time.time()
                # save_screen_shots(self.env_vis.viewer.window,f'{frame_dir}/%04d.png' % fr,  autogui = True)
                
                width, height = glfw.get_window_size(self.env_vis.viewer.window)
                data = self.env_vis._get_viewer("human").read_pixels(width, height, depth=False)
                save_image_hwc(data[::-1, :, [0,1,2]],  f'{frame_dir}/%04d.png' % fr)
                print('%d/%d, %.3f' % (fr, self.num_fr, time.time() - t0))

        # if not args.preview:
        #     out_name = f'{args.video_dir}/{args.cfg}_{"expert" if args.record_expert else args.iter}.mp4'
        #     cmd = ['ffmpeg', '-y', '-r', '30', '-f', 'image2', '-start_number', '0',
        #         '-i', f'{frame_dir}/%04d.png', '-vcodec', 'libx264', '-crf', '5', '-pix_fmt', 'yuv420p', out_name]
        #     subprocess.call(cmd)

In [4]:

vis = MyVisulizerFK(f'mocap_v2_vis.xml')
# vis.num_fr

vis.record_video()


Creating window glfw


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


0/75, 0.262
1/75, 0.261
2/75, 0.272
3/75, 0.255
4/75, 0.267
5/75, 0.269
6/75, 0.285
7/75, 0.275
8/75, 0.297
9/75, 0.279
10/75, 0.300
11/75, 0.296
12/75, 0.282
13/75, 0.275
14/75, 0.284
15/75, 0.297
16/75, 0.265
17/75, 0.276
18/75, 0.277
19/75, 0.259
20/75, 0.289
21/75, 0.264
22/75, 0.265
23/75, 0.307
24/75, 0.263
25/75, 0.289
26/75, 0.274
27/75, 0.298
28/75, 0.293
29/75, 0.269
30/75, 0.266
31/75, 0.289
32/75, 0.276
33/75, 0.272
34/75, 0.259
35/75, 0.278
36/75, 0.270
37/75, 0.267
38/75, 0.248
39/75, 0.286
40/75, 0.278
41/75, 0.272
42/75, 0.264
43/75, 0.271
44/75, 0.236
45/75, 0.276
46/75, 0.284
47/75, 0.273
48/75, 0.267
49/75, 0.261
50/75, 0.277
51/75, 0.270
52/75, 0.250
53/75, 0.274
54/75, 0.271
55/75, 0.289
56/75, 0.287
57/75, 0.283
58/75, 0.272
59/75, 0.261
60/75, 0.278
61/75, 0.275
62/75, 0.259
63/75, 0.282


NameError: name 'exit' is not defined